In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv('/content/network_data.csv')

# Assigning features and target variables
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Training and testing the model
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)




Accuracy: 0.935


In [ ]:
import os, secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes


# Defining the standard AES encryption algorithm
class AES:
    @staticmethod

    # Standard encryption with CBC mode where we are padding the message to ensure its length is a multiple of the block size
    def encrypt(message, key, iv):
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(message) + padder.finalize()
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
        return ciphertext

    @staticmethod
    # Standard decryption with AES-CBC
    def decrypt(ciphertext, key, iv):
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        unpadder = padding.PKCS7(128).unpadder()
        decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
        plaintext = unpadder.update(decrypted_padded) + unpadder.finalize()
        return plaintext


# Defining our Dynamic Encryption Algorithm
class DynamicEncryption:
    @staticmethod
    def encrypt(message, key, iv, dynamic_encryption):
        if dynamic_encryption:
          salt = os.urandom(16)

          # Using PBKDF2HMAC algorithm to strech our key using salt
          kdf = PBKDF2HMAC(
                algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000,
                backend=default_backend()
            )

          # Deriving new a key from the original key and salt
          derived_key = kdf.derive(key)

          # Adding random padding to the message
          padding_length = secrets.randbelow(16) + 1
          padded_message = message + os.urandom(padding_length) + bytes([padding_length])


          # Using GCM mode to encrypt
          cipher = Cipher(algorithms.AES(derived_key), modes.GCM(iv), backend=default_backend())
          encryptor = cipher.encryptor()
          ciphertext = encryptor.update(padded_message) + encryptor.finalize()

          return ciphertext, encryptor.tag, padding_length, salt

        else:
            return AES.encrypt(message, key, iv), None, None, None


    # Decrypting the plaintext
    @staticmethod
    def decrypt(ciphertext, key, iv, dynamic_encryption, tag=None, padding_length=None, salt = None):
        if dynamic_encryption and salt is not None:
            kdf = PBKDF2HMAC(
                algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000,
                backend=default_backend()
            )
            derived_key = kdf.derive(key)
            cipher = Cipher(algorithms.AES(derived_key), modes.GCM(iv, tag), backend=default_backend())
            decryptor = cipher.decryptor()
            decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
            plaintext = decrypted_padded[:-padding_length-1]
            return plaintext
        else:
            return AES.decrypt(ciphertext, key, iv)

def encryption_system(network_metrics, rf_model):
    #Determining the network status
    network_status = rf_model.predict([network_metrics])[0]

    user_input = input("Enter your message for encryption: ")
    message = user_input.encode()

    # Generating a random key
    key = os.urandom(32)

    # Generating a random IV of length 16 or 12 based on the network status
    if(network_status == 0):
        iv = os.urandom(16)
    else:
        iv = os.urandom(12)


    ciphertext, tag, padding_length, salt = DynamicEncryption.encrypt(message, key, iv, dynamic_encryption=network_status==1)
    plaintext = DynamicEncryption.decrypt(ciphertext, key, iv, dynamic_encryption=network_status==1, tag=tag, padding_length=padding_length, salt=salt)

    # Printing the necessary information for the user
    print("Network Status:", "Under Attack" if network_status == 1 else "Normal")

    print("Original message:", message.decode())

    print("Encrypted message:", ciphertext.hex())

    if(network_status == 1):
        print("Tag:", tag.hex())
        print("Salt:", salt.hex())

    print("Decrypted message:", plaintext.decode())


#Input the network parameters
encryption_system([300, 25], rf_model)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Enter your message for encryption: Hello
Network Status: Under Attack
Original message: Hello
Encrypted message: 2cb78798f6675f6ab17b7904144dc6c70e02341f
Tag: efedf39579b4acf8c76af1c3a0895fb5
Salt: e39d4d879cf9e1bdaf49229cdbc811d7
Decrypted message: Hello
